In [1]:
import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.math as afwMath
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
from lsst.cp.pipe.utils import (arrangeFlatsByExpTime, arrangeFlatsByExpId,
                                sigmaClipCorrection, CovFastFourierTransform)

import lsst.pipe.base.connectionTypes as cT

from lsst.ip.isr import PhotonTransferCurveDataset
from lsst.ip.isr import IsrTask

In [2]:
def getReadNoiseFromMetadata(taskMetadata, ampName):
    """Gets readout noise for an amp from ISR metadata.
    Parameters
    ----------
    taskMetadata : `list` [`lsst.pipe.base.TaskMetadata`]
                List of exposures metadata from ISR.
    ampName : `str`
        Amplifier name.
    Returns
    -------
    readNoise : `float`
        Median of the overscan readnoise in the
        post-ISR metadata of the input exposures (ADU).
        Returns 'None' if the median could not be calculated.
    """
    # Empirical readout noise [ADU] measured from an
    # overscan-subtracted overscan during ISR.
    expectedKey = f"RESIDUAL STDEV {ampName}"

    readNoises = []
    for expMetadata in taskMetadata:
        if 'isr' in expMetadata:
            overscanNoise = expMetadata['isr'][expectedKey]
        else:
            continue
        readNoises.append(overscanNoise)

    if len(readNoises):
        readNoise = np.median(np.array(readNoises))
    else:
        print("Median readout noise from ISR metadata for amp %s "
                         "could not be calculated." % ampName)
        readNoise = None

    return 

In [3]:
genCollection = "u/lgiraldo/bps_ALLCCD_20220701"
butler = dB.Butler("/repo/main/", collections=genCollection)

In [4]:
exposure_pair = [3021120700503, 3021120700504]

##These exposures have a mean of around  5000 ADU
exp1 = butler.get('cpPtcProc', detector=55, exposure=exposure_pair[0], instrument='LSSTCam')
exp2 = butler.get('cpPtcProc', detector=55, exposure=exposure_pair[1], instrument='LSSTCam')

In [5]:
metadata = butler.get('isr_metadata', instrument='LSSTCam', exposure=exposure_pair[1], detector=55)
print(metadata['isr']["RESIDUAL STDEV C00"])

3.4678581362725582


In [6]:
getReadNoiseFromMetadata(metadata, "C00")

TypeError: string indices must be integers